# Reading in MCTS (Monte Carlo Tree Search) data
### The goal is to train two RNNs.
 - Predict Probability of each of the moves being successful
 - Evaluate the value of the current board position

In [5]:
# uninstall numpy and then reinstall/update tensorflow
import pandas as pd
import numpy as np
print(np.__version__) # must not be the most recent. 1.26.4 works.
import tensorflow as tf
print(tf.__version__)

from board import Board


2.0.0



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1077, in launch

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [18]:
import pandas as pd 
print(pd.__version__) # should work with 2.2.2
from board import Board

df = pd.read_pickle('Data/playing_data.pkl')
cols = ['Board', 'Moves','Move_Probs','Reward', 'Weights']

print(df.head)
print(df.ndim, ' dimensions')
print(df.shape, ' shape')
print(df.size, ' size')

2.2.2
<bound method NDFrame.head of                                                 Board  \
0   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
1   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
2   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
3   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
4   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
5   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
6   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
7   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
8   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
9   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
10  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
11  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
12  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
13  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
14  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
15  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [..

In [19]:
# checking out to make the data is stored correctly
  
row = 12
print(df.iloc[row])
print(df.iloc[row]['inv_left'])
x = df.iloc[row]['Board']
board = Board(14)
board.show_dots = False
board.board = x
print(board)
print(df.iloc[row]['Move_Probs'])
print(df.iloc[row]['Moves'])


Board         [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...
Moves         [[7, 4, 19, 3, 18, 3], [2, 9, 17, 0, 4, 1], [1...
Move_Probs    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Reward                                                        1
Weights       [52.516605169314225, 45.70846983801697, 40.067...
Num_Sims                                                    394
mc_type                                              monteCarlo
inv_left                          [2, 4, 8, 10, 12, 13, 15, 20]
Name: 12, dtype: object
[2, 4, 8, 10, 12, 13, 15, 20]
   A B C D E F G H I J K L M N
 1 □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
 2 □ □ ▣ □ □ □ □ ▣ □ □ □ □ □ □ 
 3 □ □ ▣ ▣ □ ▣ ▣ ▣ □ □ □ □ □ □ 
 4 □ □ □ ▣ □ ▣ □ □ □ □ □ □ □ □ 
 5 ▣ ▣ ▣ □ ▣ □ □ □ □ □ □ ▣ □ □ 
 6 ▣ □ ▣ □ ▣ ▣ □ □ □ □ □ ▣ □ □ 
 7 □ □ □ □ ▣ □ □ □ □ □ □ ▣ □ □ 
 8 □ □ □ □ □ ▣ ▣ □ ▣ □ □ ▣ ▣ □ 
 9 □ □ □ □ ▣ ▣ □ □ ▣ ▣ ▣ □ □ ▣ 
10 □ □ □ □ □ ▣ □ □ □ ▣ □ □ ▣ ▣ 
11 □ □ □ □ ▣ □ ▣ ▣ ▣ □ □ □ ▣ □ 
12 □ □ □ ▣ ▣ □ ▣ □ □ □ ▣ ▣ □ □ 
13 □ □ ▣ ▣ □ □ ▣ □ 

# Working on the DCNNs
### References:
 - https://medium.com/aiguys/deep-convolutional-neural-networks-dcnns-explained-in-layman-terms-b990b2818061
  - https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/


In [30]:
# Data preprocessing
import math

df = df.sample(frac = 1)
training_ratio = 0.8
split_index = math.floor(training_ratio*(df.shape[0]))
print("Amount of training batches: ", split_index)
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

train_boards = np.array(tuple(train_df['Board'].values))
train_rewards = np.array(tuple(train_df['Reward'].values))

test_boards = np.array(tuple(test_df['Board'].values))
test_rewards = np.array(tuple(test_df['Reward'].values))

print("Shape of training boards: ", np.shape(train_boards))

Amount of training batches:  35
Shape of training boards:  (35, 14, 14)


In [2]:
# Defining the evaluation snd predictive networks
def create_policy_network(batch_size):
    # Define policy network architecture using TensorFlow or other deep learning frameworks
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(batch_size, 14, 14)),
        # Add more layers as needed
        tf.keras.layers.Dense(361, activation='softmax')
    ])
    return model

def create_value_network(batch_size):
    # Define value network architecture
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(batch_size, 14, 14)),
        # Add more layers as needed
        tf.keras.layers.Dense(1, activation='tanh')
    ])
    return model


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1077, in launch

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
# Code snippet for training the neural networks
def train_networks(policy_network, value_network, preprocessed_data):
    # Define loss functions and optimizer
    policy_loss = tf.keras.losses.CategoricalCrossentropy()
    value_loss = tf.keras.losses.MeanSquaredError()
    optimizer = tf.keras.optimizers.Adam()

    # Compile the models
    policy_network.compile(optimizer=optimizer, loss=policy_loss)
    value_network.compile(optimizer=optimizer, loss=value_loss)

    # Train the models
    policy_network.fit(preprocessed_data['input'], preprocessed_data['policy_target'], epochs=10, batch_size=32)
    value_network.fit(preprocessed_data['input'], preprocessed_data['value_target'], epochs=10, batch_size=32)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1077, in launch

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1077, in launch

AttributeError: _UFUNC_API not found

ImportError: numpy.core.umath failed to import